<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Secure RESTful APIs using Python
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:56%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# Encrypted connections

Security comes in layers. Before we can discuss authentication with JSON Web Tokens or HTTP Signatures, we need to talk about SSL security.  The goal of SSL, or actually of TLS which has superceded it to fix some weaknesses, is to have clients and servers communicate only encrypted messages.  This way, if anyone listens on the publicly routed TCP packets, all they see are random bytes that they cannot interpret.  When you use the HTTPS protocol, the "S" means that SSL encryption is used for the connection.  Ordinary web browsers usually add some sort of icon or coloring to indicate that secure URLs and connections are being used.

Let's look at a quick example of where private information could be exposed.  A very simple server, `app0.py` is running on port 5000 on my system.  This server uses plain HTTP protocol—as have those in the prior lesson.  After this, we will start using only encrypted HTTPS URLs in future examples.  

Imagine you wanted to make a request to this server.  Here it is on `localhost`, but it could equally be at a publicly accessible domain name.  In order to make this illustration simpler, the request will indicate the `identity` encoding so that compression of responses is not used.  However, keep in mind that compression is not encryption.  It can make text hard to read with the naked eye, but no harder for compression libraries, such as Python's `gzip` to expose the content.

In [1]:
import requests
headers = {"Accept-Encoding": "identity",}
resp = requests.get("http://localhost:5000/", headers=headers)
resp.text

'Secret Message!'

The problem is that unbeknownst to us, someone was monitoring the TCP packets.  For example, I ran this command to see what was being communicated (various other tools can do the same thing, and can do it on any intermediate routers between me and a server).

```bash
% sudo tcpdump -i any -Aq src port 5000
```

The captured content of this server to client communication looks like:

```
`......@...................................p....Dm..P.......
E..<..@.@.<..................5.......0.........
E..4Z.@.@....................5.y.....(.....
{.0.{.0.
E..EZ.@.@....................5.y.....9.....
{.0.{.0.HTTP/1.0 200 OK

E...Z.@.@..@.................5.y...........
{.0.{.0.Content-Type: text/html; charset=utf-8
Content-Length: 15
Server: Werkzeug/1.0.1 Python/3.8.2
Date: Sun, 18 Apr 2021 05:54:02 GMT

E..CZ.@.@..................\.5.y.....7.....
{.0.{.0.Secret Message!
E..4Z.@.@..................k.5.y.....(.....
{.0.{.0.
E..4Z.@.@..................l.5.z.....(.....
{.0.{.0.
```

Some of that is not obvious, especially seeing the dots for non-ASCII bytes.  But it is easy to see the HTTP response header, and crucially also the plaintext "Secret Message."  Let's fix that going forward.

## TLS Certificates

The general topic of generating, signing, and managing TLS certificates (formerly SSL certificates) is outside the scope of this course, but is discussed in other INE courses.  This process involves obtaining (i.e. paying) a Certificate Authority to validate public and private keys.  In very brief overview, a system of public key cryptography with trusted "root certificates" is used in order to allow clients and servers to communicate in a way that does not use "plaintext" messages to each other.

For illustration, however, we can generate a public and private key to use by the servers in this course.  The lifetime of these keys is set to be short, and will probably be expired by the time you watch this.  You can generate your own as shown below.  The server needs to have both keys, but the public key, as the name suggests, can be widely distributed to anyone.  That's all the client needs for the encrypted communication.

```bash
% openssl req -x509 -newkey rsa:4096 -nodes -out pubkey.pem -keyout private.pem -days 30
Generating a RSA private key
..........++++
.......................................++++
writing new private key to 'private.pem'
-----
You are about to be asked to enter information that will be incorporated
into your certificate request.
What you are about to enter is what is called a Distinguished Name or a DN.
There are quite a few fields but you can leave some blank
For some fields there will be a default value,
If you enter '.', the field will be left blank.
-----
Country Name (2 letter code) [AU]:US
State or Province Name (full name) [Some-State]:.
Locality Name (eg, city) []:.
Organization Name (eg, company) [Internet Widgits Pty Ltd]:INE
Organizational Unit Name (eg, section) []:.
Common Name (e.g. server FQDN or YOUR name) []:localhost
Email Address []:.
```

With the keys created, we can run this server `app1.py`, differing from `app0.py` only in the last line which uses port 5001 and configures these keys.

```python
#!/usr/bin/env python
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Secret Message!"

if __name__ == "__main__":
    app.run(ssl_context=('pubkey.pem', 'private.pem'), port=5001)
```

Now let's call the server.  Here we not only can, but must, call it using HTTPS.  For the moment, let's still insist that  no compression is used by specifying `Accept-Encoding` as above.

In [3]:
requests.packages.urllib3.disable_warnings()
resp = requests.get("https://localhost:5001/", 
                    headers=headers, verify='pubkey.pem')
resp.text

'Secret Message!'

In this case capturing the output contains no plaintext to make sense of, and no one other than the two ends has the temporary symmetric key generated to encrypt the connection.

```
% sudo tcpdump -i any -Aq src port 5001 -c8 | grep -v localhost
`.N....@...................................Z......=.P.......
E..<..@.@.<...............ym.$.......0.........
}c.I}c.I....
E..4k.@.@.................yn.$./.....(.....
}c.J}c.J
E...k.@.@.................yn.$./...........
}c.Q}c.J....z...v..V.-%=.#..u$.D^...MH...J.......1. .=:}...',...s./.....O..E...)        ........+.....3.$... /.;5......'.....&..+..V.P.S.`G.5..............'.L{..@
:.....b..CO'....a.oqSt.......Q.....C...p.H2.......M.&....ni..d.7.g.%;....../.,...{M.....A....M.[[..WkY} ...g...I\......G.b.-.w.7...T..G`.X..$g7....(......... .B.P'......f..).Z.T...\n.;.I@.......Y.|.=.q<..f..e..jH.?.+. .7..d......?...SI.,.C..........f...........=_...1..Fql.RC..m.^O#-.m/.._H5.......s.L.\b.b........^<.....5..
nd..=+K....K.T^>...|..W70.M.......]>..Xs...p{....   .....0........3N.o.P..dv.2..#.e
8..Mc!..../...=.r...V.F.............r..-.....NV{...u....p..Z}....i9.%..w.........>.'..v.6=s:...j..ZL...A
...>J<rY}|,.ZnNaD.J....pL.
```

The bytes transmitted are indistinguishable from random noise to anyone other than the connecting ends.

## Different security concerns

What TLS accomplishes is keeping the content of a connection private.  However, it does not provide any authentication of specific users.  TLS cannot, in itself, prevent anyone with the same URL from accessing the same content or service.  For that we need additional protocols.